In [1]:
from deformableConv_op import get_cnn, get_deform_cnn
from mnist import get_gen
import tensorflow as tf
import keras.backend as K
import numpy as np


from keras.models import Model
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam, SGD



Using TensorFlow backend.


In [4]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True #允许显存不预先分配，而是随时增加
sess = tf.Session(config=config)
K.set_session(sess)

batch_size = 32
n_train = 60000
n_test = 10000
steps_per_epoch = int(np.ceil(n_train / batch_size))
validation_steps = int(np.ceil(n_test / batch_size))

train_gen = get_gen(
    'train', batch_size=batch_size,
    scale=(1.0, 1.0), translate=0.0,
    shuffle=True
)
test_gen = get_gen(
    'test', batch_size=batch_size,
    scale=(1.0, 1.0), translate=0.0,
    shuffle=False
)
train_scaled_gen = get_gen(
    'train', batch_size=batch_size,
    scale=(1.0, 2.5), translate=0.2,
    shuffle=True
)
test_scaled_gen = get_gen(
    'test', batch_size=batch_size,
    scale=(1.0, 2.5), translate=0.2,
    shuffle=False
)

In [ ]:
inputs, outputs = get_cnn()
model = Model(inputs=inputs, outputs=outputs)
model.summary()
optim = Adam(1e-3)
# optim = SGD(1e-3, momentum=0.99, nesterov=True)
loss = categorical_crossentropy
model.compile(optim, loss, metrics=['accuracy'])

model.fit_generator(
    train_gen, steps_per_epoch=steps_per_epoch,
    epochs=10, verbose=1,
    validation_data=test_gen, validation_steps=validation_steps
)
model.save_weights('./models/cnn.h5')

In [ ]:
model.load_weights('./models/cnn.h5', by_name=True)

val_loss, val_acc = model.evaluate_generator(
    test_gen, steps=validation_steps
)
print('Test accuracy', val_acc)
# 0.9874

val_loss, val_acc = model.evaluate_generator(
    test_scaled_gen, steps=validation_steps
)
print('Test accuracy with scaled images', val_acc)

In [5]:
inputs, outputs = get_deform_cnn(trainable=False)
model = Model(inputs=inputs, outputs=outputs)
model.load_weights('./models/cnn.h5', by_name=True)
model.summary()
optim = Adam(5e-4)
# optim = SGD(1e-4, momentum=0.99, nesterov=True)
loss = categorical_crossentropy
model.compile(optim, loss, metrics=['accuracy'])

model.fit_generator(
    train_scaled_gen, steps_per_epoch=steps_per_epoch,
    epochs=20, verbose=1,
    validation_data=test_scaled_gen, validation_steps=validation_steps
)
# Epoch 20/20
# 1875/1875 [==============================] - 504s - loss: 0.2838 - acc: 0.9122 - val_loss: 0.2359 - val_acc: 0.9231
model.save_weights('./models/deform_cnn.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 28, 28, 1)         0         
_________________________________________________________________
conv11 (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
conv11_relu (Activation)     (None, 28, 28, 32)        0         
_________________________________________________________________
conv11_bn (BatchNormalizatio (None, 28, 28, 32)        128       
_________________________________________________________________
conv12_offset (DeformableCon (None, 28, 28, 32)        18432     
_________________________________________________________________
conv12 (Conv2D)              (None, 14, 14, 64)        18496     
_________________________________________________________________
conv12_relu (Activation)     (None, 14, 14, 64)        0         
__________

In [6]:
model.load_weights('./models/deform_cnn.h5')

val_loss, val_acc = model.evaluate_generator(
    test_scaled_gen, steps=validation_steps
)
print('Test accuracy of deformable convolution with scaled images', val_acc)
# 0.9255

val_loss, val_acc = model.evaluate_generator(
    test_gen, steps=validation_steps
)
print('Test accuracy of deformable convolution with regular images', val_acc)

('Test accuracy of deformable convolution with scaled images', 0.9681490384615384)
('Test accuracy of deformable convolution with regular images', 0.9182692307692307)


In [10]:
## 作为对比
inputs, outputs = get_cnn()
model1 = Model(inputs=inputs, outputs=outputs)
model1.summary()
optim = Adam(1e-3)
# optim = SGD(1e-3, momentum=0.99, nesterov=True)
loss = categorical_crossentropy
model1.compile(optim, loss, metrics=['accuracy'])

model1.load_weights('./models/cnn.h5', by_name=True)

val_loss, val_acc = model1.evaluate_generator(
    test_gen, steps=validation_steps
)
print('Test accuracy', val_acc)
# 0.9874

val_loss, val_acc = model1.evaluate_generator(
    test_scaled_gen, steps=validation_steps
)
print('Test accuracy with scaled images', val_acc)
# 0.5701

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 28, 28, 1)         0         
_________________________________________________________________
conv11 (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
conv11_relu (Activation)     (None, 28, 28, 32)        0         
_________________________________________________________________
conv11_bn (BatchNormalizatio (None, 28, 28, 32)        128       
_________________________________________________________________
conv12 (Conv2D)              (None, 14, 14, 64)        18496     
_________________________________________________________________
conv12_relu (Activation)     (None, 14, 14, 64)        0         
_________________________________________________________________
conv12_bn (BatchNormalizatio (None, 14, 14, 64)        256       
__________

In [11]:
deform_conv_layers = [l for l in model.layers if isinstance(l, DeformableConv)]

Xb, Yb = next(test_gen)
for l in deform_conv_layers:
    print(l)
    _model = Model(inputs=inputs, outputs=l.output)
    offsets = _model.predict(Xb)
    offsets = offsets.reshape(offsets.shape[0], offsets.shape[1], offsets.shape[2], -1, 2)
    print(offsets.min())
    print(offsets.mean())
    print(offsets.max())

ValueError: Graph disconnected: cannot obtain value for tensor Tensor("input_2:0", shape=(?, 28, 28, 1), dtype=float32) at layer "input". The following previous layers were accessed without issue: []